In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from datetime import date

In [228]:
X = pd.read_csv("dataset/dengue_features_train.csv")
test = pd.read_csv("dataset/dengue_features_test.csv")
y = pd.read_csv("dataset/dengue_labels_train.csv")

In [229]:
X.head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
5,sj,1990,23,1990-06-04,NaN,0.174850,0.254314,0.181743,9.58,299.630000,...,26.49,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,...,38.60,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,...,30.00,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,...,37.51,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1
9,sj,1990,27,1990-07-02,NaN,0.121550,0.160683,0.202567,14.41,300.154286,...,28.40,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1


In [230]:
y = y.drop(['city', 'year', 'weekofyear'], axis = 1)

In [231]:
X['week_start_date'] = pd.to_datetime(X['week_start_date'])
X['days_since_week_started'] = X['week_start_date'].apply(lambda x:(pd.Timestamp('today') - x).days)
X.drop(['week_start_date'], inplace=True, axis=1)

In [232]:
city_encoding = {"city" :  {"sj" : 0, "iq" : 1}}
X = X.replace(city_encoding)
X.head()

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,days_since_week_started
0,0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,11305
1,0,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,11298
2,0,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,11291
3,0,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,11284
4,0,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,11277


In [233]:
X['ndvi_ne'] = X['ndvi_ne'].fillna((X['ndvi_ne'].mean()))
X['ndvi_nw'] = X['ndvi_nw'].fillna((X['ndvi_nw'].mean()))
X['ndvi_se'] = X['ndvi_se'].fillna((X['ndvi_se'].mean()))
X['ndvi_sw'] = X['ndvi_sw'].fillna((X['ndvi_sw'].mean()))
X['precipitation_amt_mm'] = X['precipitation_amt_mm'].fillna((X['precipitation_amt_mm'].mean()))
X['reanalysis_air_temp_k'] = X['reanalysis_air_temp_k'].fillna((X['reanalysis_air_temp_k'].mean()))
X['reanalysis_avg_temp_k'] = X['reanalysis_avg_temp_k'].fillna((X['reanalysis_avg_temp_k'].mean()))
X['reanalysis_dew_point_temp_k'] = X['reanalysis_dew_point_temp_k'].fillna((X['reanalysis_dew_point_temp_k'].mean()))
X['reanalysis_max_air_temp_k'] = X['reanalysis_max_air_temp_k'].fillna((X['reanalysis_max_air_temp_k'].mean()))
X['reanalysis_min_air_temp_k'] = X['reanalysis_min_air_temp_k'].fillna((X['reanalysis_min_air_temp_k'].mean()))
X['reanalysis_precip_amt_kg_per_m2'] = X['reanalysis_precip_amt_kg_per_m2'].fillna((X['reanalysis_precip_amt_kg_per_m2'].mean()))
X['reanalysis_relative_humidity_percent'] = X['reanalysis_relative_humidity_percent'].fillna((X['reanalysis_relative_humidity_percent'].mean()))
X['reanalysis_sat_precip_amt_mm'] = X['reanalysis_sat_precip_amt_mm'].fillna((X['reanalysis_sat_precip_amt_mm'].mean()))
X['reanalysis_specific_humidity_g_per_kg'] = X['reanalysis_specific_humidity_g_per_kg'].fillna((X['reanalysis_specific_humidity_g_per_kg'].mean()))
X['reanalysis_tdtr_k'] = X['reanalysis_tdtr_k'].fillna((X['reanalysis_tdtr_k'].mean()))
X['station_avg_temp_c'] = X['station_avg_temp_c'].fillna((X['station_avg_temp_c'].mean()))
X['station_diur_temp_rng_c'] = X['station_diur_temp_rng_c'].fillna((X['station_diur_temp_rng_c'].mean()))
X['station_max_temp_c'] = X['station_max_temp_c'].fillna((X['station_max_temp_c'].mean()))
X['station_min_temp_c'] = X['station_min_temp_c'].fillna((X['station_min_temp_c'].mean()))
X['station_precip_mm'] = X['station_precip_mm'].fillna((X['station_precip_mm'].mean()))

In [234]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   1456 non-null   int64  
 1   year                                   1456 non-null   int64  
 2   weekofyear                             1456 non-null   int64  
 3   ndvi_ne                                1456 non-null   float64
 4   ndvi_nw                                1456 non-null   float64
 5   ndvi_se                                1456 non-null   float64
 6   ndvi_sw                                1456 non-null   float64
 7   precipitation_amt_mm                   1456 non-null   float64
 8   reanalysis_air_temp_k                  1456 non-null   float64
 9   reanalysis_avg_temp_k                  1456 non-null   float64
 10  reanalysis_dew_point_temp_k            1456 non-null   float64
 11  rean

In [235]:
test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [236]:
city = test.city

In [237]:
test['week_start_date'] = pd.to_datetime(test['week_start_date'])
test['days_since_week_started'] = test['week_start_date'].apply(lambda x:(pd.Timestamp('today') - x).days)
test.drop(['week_start_date'], inplace=True, axis=1)

In [238]:
city_encoding = {"city" :  {"sj" : 0, "iq" : 1}}
test = test.replace(city_encoding)

In [239]:
test['ndvi_ne'] = test['ndvi_ne'].fillna((test['ndvi_ne'].mean()))
test['ndvi_nw'] = test['ndvi_nw'].fillna((test['ndvi_nw'].mean()))
test['ndvi_se'] = test['ndvi_se'].fillna((test['ndvi_se'].mean()))
test['ndvi_sw'] = test['ndvi_sw'].fillna((test['ndvi_sw'].mean()))
test['precipitation_amt_mm'] = test['precipitation_amt_mm'].fillna((test['precipitation_amt_mm'].mean()))
test['reanalysis_air_temp_k'] = test['reanalysis_air_temp_k'].fillna((test['reanalysis_air_temp_k'].mean()))
test['reanalysis_avg_temp_k'] = test['reanalysis_avg_temp_k'].fillna((test['reanalysis_avg_temp_k'].mean()))
test['reanalysis_dew_point_temp_k'] = test['reanalysis_dew_point_temp_k'].fillna((test['reanalysis_dew_point_temp_k'].mean()))
test['reanalysis_max_air_temp_k'] = test['reanalysis_max_air_temp_k'].fillna((test['reanalysis_max_air_temp_k'].mean()))
test['reanalysis_min_air_temp_k'] = test['reanalysis_min_air_temp_k'].fillna((test['reanalysis_min_air_temp_k'].mean()))
test['reanalysis_precip_amt_kg_per_m2'] =test['reanalysis_precip_amt_kg_per_m2'].fillna((test['reanalysis_precip_amt_kg_per_m2'].mean()))
test['reanalysis_relative_humidity_percent'] = test['reanalysis_relative_humidity_percent'].fillna((test['reanalysis_relative_humidity_percent'].mean()))
test['reanalysis_sat_precip_amt_mm'] = test['reanalysis_sat_precip_amt_mm'].fillna((test['reanalysis_sat_precip_amt_mm'].mean()))
test['reanalysis_specific_humidity_g_per_kg'] = test['reanalysis_specific_humidity_g_per_kg'].fillna((test['reanalysis_specific_humidity_g_per_kg'].mean()))
test['reanalysis_tdtr_k'] = test['reanalysis_tdtr_k'].fillna((test['reanalysis_tdtr_k'].mean()))
test['station_avg_temp_c'] = test['station_avg_temp_c'].fillna((test['station_avg_temp_c'].mean()))
test['station_diur_temp_rng_c'] = test['station_diur_temp_rng_c'].fillna((test['station_diur_temp_rng_c'].mean()))
test['station_max_temp_c'] = test['station_max_temp_c'].fillna((test['station_max_temp_c'].mean()))
test['station_min_temp_c'] = test['station_min_temp_c'].fillna((test['station_min_temp_c'].mean()))
test['station_precip_mm'] = test['station_precip_mm'].fillna((test['station_precip_mm'].mean()))

In [240]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   416 non-null    int64  
 1   year                                   416 non-null    int64  
 2   weekofyear                             416 non-null    int64  
 3   ndvi_ne                                416 non-null    float64
 4   ndvi_nw                                416 non-null    float64
 5   ndvi_se                                416 non-null    float64
 6   ndvi_sw                                416 non-null    float64
 7   precipitation_amt_mm                   416 non-null    float64
 8   reanalysis_air_temp_k                  416 non-null    float64
 9   reanalysis_avg_temp_k                  416 non-null    float64
 10  reanalysis_dew_point_temp_k            416 non-null    float64
 11  reanal

In [241]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [265]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=220, min_samples_split=2, max_depth=100)
model.fit(X_train, y_train.values.ravel())
y_pred = model.predict(X_val)
mean_absolute_error(y_val, y_pred)

13.15859277708593

In [257]:
from sklearn.tree import DecisionTreeRegressor

model1 = DecisionTreeRegressor(criterion="mae", max_depth=10, min_samples_split=10, max_features="auto")
model1.fit(X_train, y_train.values.ravel())
y_pred1 = model1.predict(X_val)
mean_absolute_error(y_val, y_pred1)

12.595890410958905

In [336]:
from sklearn.ensemble import GradientBoostingRegressor

model3 = GradientBoostingRegressor(loss="ls", n_estimators=170, criterion="mse", subsample=0.9, min_samples_leaf=4, max_depth=4)
model3.fit(X_train, y_train.values.ravel())
y_pred3 = model3.predict(X_val)
mean_absolute_error(y_val, y_pred3)

10.391080571071006

In [360]:
pred = model3.predict(test)

In [362]:
pred = pred.astype(int)

In [366]:
pred

array([  7,   1,   4,  10,  18,  20,  19,  14,  28,  54,  68,  59,  73,
        81,  86, 102,  39,  80,  95, 111,  88,  30,  38,  80,  40,  37,
        37,  38,  37,  35,  37,  12,  13,  28,  11,  18,  17,  25,  18,
        16,  15,  13,  13,   9,   7,   4,   4,   2,   2,   2,   1,   3,
         2,   7,   2,   2,  14,  20,  12,  18,  14,  33,  77,  81,  92,
        97,  98,  79, 105,  47,  98,  30,  30,  79, 108, 107, 103,  70,
        42,  30,  26,  33,  30,  30,  25,  23,  29,  14,  14,  14,  12,
        16,  12,  13,  12,   9,  20,  11,   4,   6,  18,   3,   1,   5,
        18,  17,  17,  15,  17,  14,  90,  82,  15,  54,  90, 113, 101,
        30,  85, 109,  92,  33, 121, 108, 109,  89,  82,  92,  96,  33,
        34,  33,  36,  17,  28,  10,   8,  12,   8,   8,  12,  10,  10,
        14,  11,   8,   7,   9,   9,   2,   7,   2,   3,   1,   1,   2,
         1,   2,   5,   3,   6,  17,  22,  29, 117,  83,  23,  65,  73,
       118,  68,  39,  84,  51, 101,  84,  81,  21,  69,  73,  8

In [364]:
output = pd.DataFrame({'city': city,
                       'year': test.year,
                       'weekofyear': test.weekofyear,
                       'total_cases': pred})

In [365]:
output.to_csv('submission.csv', index=False)